In [3]:
import requests
import urllib

Onderstaande code is de basis connectie met het API, er kunnen parameters toegevoegd worden om meer gespecificeerde informatie te verkrijgen. Op deze link staan alle parameters die we toe zouden kunnen voegen:
https://developer.yummly.com/documentation

In [4]:
response = requests.get("http://api.yummly.com/v1/api/recipes?_app_id=2b082bfe&_app_key=905dfe0267500a98d67737f446479c11")

In [5]:
mediterranean_main_recipes_15min = requests.get("http://api.yummly.com/v1/api/recipes?_app_id=2b082bfe&_app_key=905dfe0267500a98d67737f446479c11&allowedCuisine[]=cuisine^cuisine-mediterranean&allowedCuisine[]=cuisine^cuisine-italian&allowedCuisine[]=cuisine^cuisine-spanish&maxTotalTimeInSeconds=600&allowedCourse[]=course^course-Main Dishes").json()
# converted to json to be sure that the right json format is being used.  
# Recipes which are either itlian, spanish or meditarean, which are a main course and take less than 15 minutes to cook. 

In [147]:
from rdflib import Graph, Namespace, URIRef, Literal, XSD, RDF

# create an RDF graph
g = Graph()

YUM = Namespace("http://yummly.com/")
FO = Namespace("http://purl.org/ontology/fo/")
OWL = Namespace("http://www.w3.org/2002/07/owl#")

g.bind('yum', YUM)
g.bind('fo', FO)
g.bind('owl', OWL)

In [148]:
print mediterranean_main_recipes_15min['matches'][0].keys()

[u'flavors', u'rating', u'totalTimeInSeconds', u'ingredients', u'smallImageUrls', u'sourceDisplayName', u'recipeName', u'attributes', u'id', u'imageUrlsBySize']


In [149]:
ingredient_list = []
for recipe in mediterranean_main_recipes_15min['matches']:
    for ingredient in recipe['ingredients']:
        ingredient_list.append(ingredient)
        
recipe_name_list = []
for recipe in mediterranean_main_recipes_15min['matches']:
    recipe_name_list.append(recipe['recipeName'])
    
recipe_ingredient_dict = {}
for recipe in mediterranean_main_recipes_15min['matches']:
    recipe_ingredient_dict[recipe['recipeName']] = recipe['ingredients']

cooking_time_list = []
for recipe in mediterranean_main_recipes_15min['matches']:
    cooking_time_list.append(recipe['totalTimeInSeconds'])

In [150]:
recipe_ingredient_dict

{u'Baba Ghanoush Topped with Lamb and Yogurt': [u'pitas',
  u'smoked paprika',
  u'parsley',
  u'yogurt',
  u'feta cheese',
  u'lamb',
  u'cumin',
  u'tahini',
  u'garlic',
  u'eggplant'],
 u'Braised Italian Pot Roast': [u'shoulder roast',
  u'sweet onion',
  u'italian seasoning',
  u'garlic',
  u'red wine'],
 u'Easy Garlic and Lemon Shrimp': [u'jumbo shrimp',
  u'olive oil',
  u'butter',
  u'minced garlic',
  u'lemon juice',
  u'lemon zest',
  u'sea salt'],
 u'Easy Peasy Skillet Lasagna': [u'extra-virgin olive oil',
  u'onions',
  u'mushrooms',
  u'garlic cloves',
  u'diced tomatoes',
  u'tomato sauce',
  u'basil leaves',
  u'black pepper',
  u'sea salt',
  u'ricotta cheese',
  u'part-skim mozzarella cheese',
  u'parmesan cheese',
  u'lasagna noodles',
  u'parsley'],
 u'Italian Chicken Panini': [u'chicken breasts',
  u'pesto',
  u'Italian bread',
  u'red onion',
  u'roasted red peppers',
  u'fresh mozzarella cheese',
  u'cooking spray'],
 u'Macaroni with Greens, Lemon, and Parmesan': 

In [153]:
for ingredient in ingredient_list:
    ingredientNoSpace = ingredient.replace(" ", "_")
    g.add((YUM[ingredientNoSpace], RDF.type, FO["Ingredient"]))
    g.add((YUM[ingredientNoSpace], RDF.type, OWL['NamedIndividual'] ))

In [154]:
print g.serialize(format='turtle')

@prefix fo: <http://purl.org/ontology/fo/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix yum: <http://yummly.com/> .

yum:Italian_bread a fo:Ingredient,
        owl:NamedIndividual .

yum:angel_hair a fo:Ingredient,
        owl:NamedIndividual .

yum:baby_spinach a fo:Ingredient,
        owl:NamedIndividual .

yum:basil_leaves a fo:Ingredient,
        owl:NamedIndividual .

yum:black_pepper a fo:Ingredient,
        owl:NamedIndividual .

yum:butter a fo:Ingredient,
        owl:NamedIndividual .

yum:cherry_tomatoes a fo:Ingredient,
        owl:NamedIndividual .

yum:chicken_breasts a fo:Ingredient,
        owl:NamedIndividual .

<http://yummly.com/clove_garlic,_fine_chop> a fo:Ingredient,
        owl:NamedIndividual .

yum:cooking_spray a fo:Ingredient,
  

In [156]:
for recipe in recipe_name_list:
    recipeNoSpace = recipe.replace(" ", "_")
    cleanRecipe = recipeNoSpace.replace("{","")
    cleanRecipe = cleanRecipe.replace("}","")
    cleanRecipe = cleanRecipe.replace(",", "")
    cleanRecipe = cleanRecipe.replace("-","_")
    g.add((YUM[cleanRecipe], RDF.type, FO["Recipe"]))
    g.add((YUM[cleanRecipe], RDF.type, OWL['NamedIndividual'] ))

In [157]:
print g.serialize(format='turtle')

@prefix fo: <http://purl.org/ontology/fo/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix yum: <http://yummly.com/> .

<http://yummly.com/5_ingredient_Italian_Caprese_Pasta> a fo:Recipe,
        owl:NamedIndividual .

yum:Baba_Ghanoush_Topped_with_Lamb_and_Yogurt a fo:Recipe,
        owl:NamedIndividual .

yum:Braised_Italian_Pot_Roast a fo:Recipe,
        owl:NamedIndividual .

yum:Easy_Garlic_and_Lemon_Shrimp a fo:Recipe,
        owl:NamedIndividual .

yum:Easy_Peasy_Skillet_Lasagna a fo:Recipe,
        owl:NamedIndividual .

yum:Italian_Chicken_Panini a fo:Recipe,
        owl:NamedIndividual .

yum:Italian_bread a fo:Ingredient,
        owl:NamedIndividual .

yum:Macaroni_with_Greens_Lemon_and_Parmesan a fo:Recipe,
        owl:NamedIndividual .

yum:Medi

In [159]:
g.add((FO['hasIngredient'], RDF.type, OWL['ObjectProperty']))

In [160]:
print g.serialize(format='turtle')

@prefix fo: <http://purl.org/ontology/fo/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix yum: <http://yummly.com/> .

fo:hasIngredient a owl:ObjectProperty .

<http://yummly.com/5_ingredient_Italian_Caprese_Pasta> a fo:Recipe,
        owl:NamedIndividual .

yum:Baba_Ghanoush_Topped_with_Lamb_and_Yogurt a fo:Recipe,
        owl:NamedIndividual .

yum:Braised_Italian_Pot_Roast a fo:Recipe,
        owl:NamedIndividual .

yum:Easy_Garlic_and_Lemon_Shrimp a fo:Recipe,
        owl:NamedIndividual .

yum:Easy_Peasy_Skillet_Lasagna a fo:Recipe,
        owl:NamedIndividual .

yum:Italian_Chicken_Panini a fo:Recipe,
        owl:NamedIndividual .

yum:Italian_bread a fo:Ingredient,
        owl:NamedIndividual .

yum:Macaroni_with_Greens_Lemon_and_Parmesan a fo:Recipe